## args kwargs

args -> positional arguments only

In [61]:
def adder(*num):
    sum = 0
    
    for n in num:
        sum = sum + n
        
    print('sum', sum)
    
adder(3,5)
adder(4,5,6,7)

sum 8
sum 22


**kwargs -->  keyword arguments only

In [66]:
def intro(**data):
    print("\n data type of argument", type(data))
    
    for key, value in data.items():
        print('{} is {}'.format(key,value))


intro(Firstname="Sita", Lastname="Sharma", Age=22, Phone=1234567890)
intro(Firstname="John", Lastname="Wood", Email="johnwood@nomail.com", Country="Wakanda", Age=25, Phone=9876543210)


 data type of argument <class 'dict'>
Firstname is Sita
Lastname is Sharma
Age is 22
Phone is 1234567890

 data type of argument <class 'dict'>
Firstname is John
Lastname is Wood
Email is johnwood@nomail.com
Country is Wakanda
Age is 25
Phone is 9876543210


## Yield n Generators

## Context Manager

when to use: 
- need to setup , teardown ressources during use (i.e. open close db connections )

<b> with </b>
- with keyword is used when working with unmanaged resources (like file streams).
- ensure that a resource is "cleaned up" when the code that uses it finishes running, even if exceptions are thrown. It provides 'syntactic sugar' for try/finally blocks.
- https://stackoverflow.com/questions/1369526/what-is-the-python-keyword-with-used-for


- https://www.youtube.com/watch?v=-aKFBoZpiqA

### via class

In [6]:
class Open_File():
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        
    def __enter__(self):
        self.file= open(self.filename, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_val, traceback):
        self.file.close()

In [7]:
with Open_File('sample.txt', 'w') as f:
    f.write('Lorem ipsum dolor sit amet, consectetur adipiscing elit.')

print(f.closed) # check that closed after written

True


### via function

In [18]:
from contextlib import contextmanager

@contextmanager
def open_file(file, mode):
    try:
        f = open(file, mode)
        yield f
    finally:
        f.close()

In [19]:
with open_file('sample1.txt', 'w') as f:
    f.write('Lorem ipsum dolor sit amet, consectetur adipiscing elit.')

print(f.closed)

True


### Practical example

#### without context manager

In [24]:
import os
from contextlib import contextmanager

# get current working directory
cwd = os.getcwd()  
os.chdir('sample-dir1')
# see files in those directory
print(os.listdir()) 
# change back to current directory
os.chdir(cwd)


#repeat
cwd = os.getcwd()
os.chdir('sample-dir2')
print(os.listdir())
os.chdir(cwd)

['to-do.txt', '.ipynb_checkpoints']
['testing123.txt', '.ipynb_checkpoints']


#### with context manager

In [26]:
import os
from contextlib import contextmanager

@contextmanager
def change_dir(destination):
    try:
        cwd = os.getcwd()
        os.chdir(destination)
        yield
    finally:
        os.chdir(cwd)


with change_dir('sample-dir1'):
    print(os.listdir())

with change_dir('sample-dir2'):
    print(os.listdir())

['to-do.txt', '.ipynb_checkpoints']
['testing123.txt', '.ipynb_checkpoints']


## Closure

### when is there closure:
- have a nested function 
- The nested function must refer to a value defined in the enclosing function (sub function)
- <b> The enclosing function must return the nested function. </b>
- https://www.programiz.com/python-programming/closure

In [33]:
def print_msg(msg):
    def printer():
        print(msg)
        
    return printer

another = print_msg("hello")
another()

hello


The print_msg() function was called with the string "Hello" and the returned function was bound to the name another. On calling another(), the message was still remembered although we had already finished executing the print_msg() function. Function scope ended, variable shld be removed. Unless--> closure

In [34]:
del print_msg
another()

hello


returned function still works when original function was deleted !

### when to use:

- can avoid the use of global values and provides some form of data hiding. It can also provide an object oriented solution to the problem.

- When there are few methods (one method in most cases) to be implemented in a class, closures can provide an alternate and more elegant solution. But when the number of attributes and methods get larger, it's better to implement a class.

In [43]:
def make_multiplier_of(n):
    def multiplier(x):
        return x * n
    return multiplier


# Multiplier of 3
times3 = make_multiplier_of(3)

# Output: 27
print(times3(9))

27


### closure attribute

attribute that returns tuple of cell objects if it is closure function

In [37]:
make_multiplier_of.__closure__

In [38]:
times3.__closure__

(<cell at 0x7fd48c4436d0: int object at 0x954e80>,)

In [41]:
# cell_contents stores closed value
times3.__closure__[0].cell_contents

3

## Decorators

- add functionality to function (metaprogramming)
- Nests functions inside decorator function

In [44]:
def divide(a, b):
    return a/b

In [45]:
divide(2,5)

0.4

In [59]:
def smart_divide(func):
    def inner(a, b):
        print('i am going to divide', a , 'and', b)
        if b ==0:
            print('opps canot divide')
            return
        return func(a,b)
    return inner

@smart_divide
def divide(a,b):
    print(a,b)

In [60]:
divide(2,0)

i am going to divide 2 and 0
opps canot divide


### Chaining Decorators

In [71]:
def star(func):
    def inner(*args, **kwargs):
        print("*" * 30)
        func(*args, **kwargs)
        print("*" * 30)
    return inner


def percent(func):
    def inner(*args, **kwargs):
        print("%" * 30)
        func(*args, **kwargs)
        print("%" * 30)
    return inner


@star
@percent
def printer(msg):
    print(msg)


printer("Hello")

******************************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Hello
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
******************************


### @property decorator

https://www.programiz.com/python-programming/property

In [74]:
# Basic method of setting and getting attributes in Python
class Celsius:
    def __init__(self, temperature=0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32


# Create a new object
human = Celsius()

# Set the temperature
human.temperature = 37

# Get the temperature attribute
print(human.temperature)

# Get the to_fahrenheit method
print(human.to_fahrenheit())

37
98.60000000000001


#### use setters to restrict values to be passed in (>-273.15 temperature)

In [76]:
# Making Getters and Setter methods
class Celsius:
    def __init__(self, temperature=0):
        self.set_temperature(temperature)

    def to_fahrenheit(self):
        return (self.get_temperature() * 1.8) + 32

    # getter method
    def get_temperature(self):
        return self._temperature

    # setter method
    def set_temperature(self, value):
        if value < -273.15:
            raise ValueError("Temperature below -273.15 is not possible.")
        self._temperature = value


# Create a new object, set_temperature() internally called by __init__
human = Celsius(37)

# Get the temperature attribute via a getter
print(human.get_temperature())

# Get the to_fahrenheit method, get_temperature() called by the method itself
print(human.to_fahrenheit())

# new constraint implementation
human.set_temperature(-300)

# Get the to_fahreheit method
print(human.to_fahrenheit())

37
98.60000000000001


ValueError: Temperature below -273.15 is not possible.

However, the bigger problem with the above update is that all the programs that implemented our previous class have to modify their code from obj.temperature to obj.get_temperature() and all expressions like obj.temperature = val to obj.set_temperature(val).

This refactoring can cause problems while dealing with hundreds of thousands of lines of codes.

All in all, our new update was not backwards compatible. This is where <b> @property </b> comes to rescue.

<b> property(fget=None, fset=None, fdel=None, doc=None) </b> returns property object

- fget --> <b> func </b> to get value of attribute
- fset --> func to set value of attribute
- fdel --> func to delete value of attribute
- doc --> string (comment)

In [79]:
# using property class
class Celsius:
    def __init__(self, temperature=0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    # getter
    def get_temperature(self):
        print("Getting value...")
        return self._temperature

    # setter
    def set_temperature(self, value):
        print("Setting value...")
        if value < -273.15:
            raise ValueError("Temperature below -273.15 is not possible")
        self._temperature = value

    # creating a property object
    temperature = property(get_temperature, set_temperature)
    

human = Celsius(37)
# set_temperature() was called when creating object too, as __init__() is called

print(human.temperature) 
# auto calls get_temperature() instead of __dict__ lookup

print(human.to_fahrenheit())
# assign value to temperature will auto call set_temperature()

human.temperature = -300

Setting value...
Getting value...
37
Getting value...
98.60000000000001
Setting value...


ValueError: Temperature below -273.15 is not possible

#### using property decorator

- reuse temperature name for getter and setter
- @property makes temperature same as method temperature.getter(get_temperature)
- @temperature.setter same as the method temperature.setter(set_temperature) 

In [85]:
# Using @property decorator
class Celsius:
    def __init__(self, temperature=0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    @property
    def temperature(self):
        print("Getting value...")
        return self._temperature

    @temperature.setter
    def temperature(self, value):
        print("Setting value...")
        if value < -273.15:
            raise ValueError("Temperature below -273 is not possible")
        self._temperature = value


# create an object
human = Celsius(37)
print(human.temperature)
print(human.to_fahrenheit())

Setting value...
Getting value...
37
Getting value...
98.60000000000001


In [88]:
coldest_thing = Celsius(-300)

Setting value...


ValueError: Temperature below -273 is not possible

## special methods

### \__repr__

\__repr__ used to print string of object

- Implement \__repr__ for any class you implement. This should be second nature. 
- Implement \__str__ if you think it would be useful to have a string version which errs on the side of readability.

https://stackoverflow.com/questions/1436703/what-is-the-difference-between-str-and-repr

In [7]:
# A simple Person class

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __repr__(self):
        rep = 'the Person is ' + self.name + ',' + ' aged ' + str(self.age) 
        return rep


# Let's make a Person object and print the results of repr()

person = Person("John", 20)
print(repr(person))

the Person is John, aged 20
